In [9]:
# importing neccessary packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [43]:

df= pd.read_csv("Processed_data.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,labels
0,-2.525820,1,2,-0.149347,-1.782424,0,2,1.930425,0,0.0,0
1,-2.398061,1,2,-0.715089,-0.088855,0,0,0.876100,0,0.0,0
2,-2.270301,0,1,2.113621,-0.180399,0,1,1.297830,0,0.0,0
3,-2.142542,0,2,-1.846574,-0.455032,0,1,0.454370,0,0.0,0
4,-2.014782,0,2,-1.563703,-0.775437,0,0,1.086965,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
256,0.540408,1,4,1.547879,1.253794,0,0,-1.907319,1,2.5,1
257,0.795927,0,3,-0.149347,0.689271,0,1,-1.654281,1,0.0,1
258,1.051446,1,4,1.265008,1.421625,1,0,0.454370,1,3.0,1
259,1.306965,0,2,2.679363,2.199751,0,0,-1.232551,1,1.0,1


In [44]:
X_arr, Y_arr= df.drop(columns='labels'), df['labels']


In [85]:
# Models to be used: Logistic regression, Decision Trees, Random Forest
X_train, X_test, Y_train, Y_test= train_test_split(X_arr, Y_arr, test_size=0.2)
models= ['Logistic Regression', 'Naive_Bayes','Decision Tree','Random Forest']

model1= LogisticRegression()
model2= DecisionTreeClassifier()
model3= RandomForestClassifier()
model4= GaussianNB()

param_Trees={'criterion':['gini','entropy','log_loss'],'ccp_alpha':[0.1,0.5,1,3,5,10]}
param_RF={'criterion':['gini','entropy','log_loss'],'ccp_alpha':[0.1,0.5,1,3,5,10], 'n_estimators':[1,5,10,20,100]}
cv_tree= GridSearchCV(model2, param_grid=param_Trees, cv=5)
cv_RF= GridSearchCV(model3, param_grid=param_RF, cv=5 )


In [86]:
cv_tree.fit(X_train, Y_train)
cv_RF.fit(X_train, Y_train)
model1.fit(X_train, Y_train)
model4.fit(X_train, Y_train)

tree_pred= cv_tree.predict(X_test)
RF_pred= cv_RF.predict(X_test)
log_pred= model1.predict(X_test)
NB_pred= model4.predict(X_test)


In [87]:
def KFold_CV(X,Y):
    kf= KFold(n_splits=5)
    model1= LogisticRegression()
    model4= GaussianNB()
    score_Log_reg,score_NB=[],[]

    for train_index, test_index in kf.split(X,Y):
        X_train, Y_train = X.iloc[train_index], Y.iloc[train_index]
        X_test, Y_test= X.iloc[test_index], Y.iloc[test_index]
        model1.fit(X_train, Y_train)
        model4.fit(X_train, Y_train)

        Y1,Y4= model1.predict(X_test), model4.predict(X_test)
        score_Log_reg.append(accuracy_score(Y_test, Y1))
        score_NB.append(accuracy_score(Y_test, Y4))

    return [np.array(score_Log_reg), np.array(score_NB)]

score_log_reg, score_NB= KFold_CV(X_train,Y_train)
Final_score= [float(np.mean(score_log_reg)), float(np.mean(score_NB)), float(cv_tree.best_score_), float(cv_RF.best_score_)]
print("final cv scores",Final_score)
print('models',models)


final cv scores [0.8076655052264808, 0.8220673635307781, 0.7886178861788619, 0.8221835075493612]
models ['Logistic Regression', 'Naive_Bayes', 'Decision Tree', 'Random Forest']


In [88]:
acc_log_reg= accuracy_score(Y_test,log_pred)
acc_NB= accuracy_score(Y_test,NB_pred)
acc_trees= accuracy_score(Y_test,tree_pred)
acc_RF= accuracy_score(Y_test,RF_pred)


print('final test errors are', acc_log_reg, acc_NB, acc_trees, acc_RF)
print('models',models)


final test errors are 0.8679245283018868 0.8113207547169812 0.7735849056603774 0.7924528301886793
models ['Logistic Regression', 'Naive_Bayes', 'Decision Tree', 'Random Forest']
